# MNIST

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

# TODO: 不使用sklearn的OneHotEncoder

# 加载MNIST数据集
xs, labels = fetch_openml("mnist_784", version=1, return_X_y=True)
xs, labels  = xs[:5000] / 255, labels.astype("uint8")[:5000]
# 转换为one-hot
ohe = OneHotEncoder(sparse=False)
ys = ohe.fit_transform(labels.reshape(-1, 1))

print(xs.shape, ys.shape)
print(ys[0])

(5000, 784) (5000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [3]:
# 训练
from mflow import core, ops, opts, lays

# 超参数
lr = 0.001
epoch = 30
batch_size = 64

with core.NameScope("MNIST"):
    # 初始化变量
    x = core.Variable(size=(784, 1), trainable=False)
    y = core.Variable(size=(10, 1), trainable=False)
    # 模型定义
    hidden_1 = lays.Linear(x, 784, 100, "ReLU")
    pred = lays.Linear(hidden_1, 100, 10, None)
    predicter = ops.SoftMax(pred)
    loss = ops.loss.CrossEntropyWithSoftMax(pred, y)
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, one_hot) in enumerate(zip(xs, ys)):
            x.setValue(np.mat(feat).T)
            y.setValue(np.mat(one_hot).T)
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in xs:
            x.setValue(np.mat(feat).T)
            predicter.forward()
            preds.append(predicter.value.A.ravel())  # 结果
        preds = np.array(preds).argmax(axis=1)
        acc = (labels == preds).astype("uint8").sum() / len(xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.866.
Epoch: 2, acc: 0.913.
Epoch: 3, acc: 0.923.
Epoch: 4, acc: 0.931.
Epoch: 5, acc: 0.939.
Epoch: 6, acc: 0.947.
Epoch: 7, acc: 0.953.
Epoch: 8, acc: 0.958.
Epoch: 9, acc: 0.963.
Epoch: 10, acc: 0.967.
Epoch: 11, acc: 0.972.
Epoch: 12, acc: 0.974.
Epoch: 13, acc: 0.977.
Epoch: 14, acc: 0.981.
Epoch: 15, acc: 0.982.
Epoch: 16, acc: 0.984.
Epoch: 17, acc: 0.986.
Epoch: 18, acc: 0.987.
Epoch: 19, acc: 0.989.
Epoch: 20, acc: 0.990.
Epoch: 21, acc: 0.992.
Epoch: 22, acc: 0.994.
Epoch: 23, acc: 0.995.
Epoch: 24, acc: 0.996.
Epoch: 25, acc: 0.997.
Epoch: 26, acc: 0.998.
Epoch: 27, acc: 0.999.
Epoch: 28, acc: 0.999.
Epoch: 29, acc: 0.999.
Epoch: 30, acc: 0.999.
